In [2]:
school = "bc"

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
with open("links.txt") as f:
    text = f.read()
text = text.replace('"', '')
all_schools = {k[0].strip(): k[1].strip() for k in (school.split('-') for school in text.split('\n'))}

In [5]:
all_schools

{'Boston College': 'https://bceagles.com/index.aspx',
 'Clemson University': 'https://clemsontigers.com/',
 'Duke University': 'https://goduke.com/index.aspx',
 'Florida State University': 'https://seminoles.com/',
 'Georgia Institute of Technology (Georgia Tech)': 'https://ramblinwreck.com/',
 'University of Louisville': 'https://gocards.com/',
 'University of Miami': 'https://miamihurricanes.com/',
 'University of North Carolina at Chapel Hill': 'https://goheels.com/',
 'North Carolina State University': 'https://gopack.com/',
 'University of Notre Dame': 'https://fightingirish.com/',
 'University of Pittsburgh': 'https://pittsburghpanthers.com/index.aspx',
 'Syracuse University': 'https://cuse.com/',
 'University of Virginia': 'https://virginiasports.com/',
 'Virginia Polytechnic Institute and State University (Virginia Tech)': 'https://hokiesports.com/index.aspx',
 'Wake Forest University': 'https://godeacs.com/'}

In [124]:
def get_roster(link):
    data = pd.read_html(link)[2]
    if 'Full Name' in data.columns:
        data = data.rename(columns = {"Full Name": "Name"})
    if 'Hometown / High School' in data.columns:
        between = data['Hometown / High School'].str.extract(r'^/ (?P<HighSchool>.+)/(?P<Hometown>.+)$|^(?P<HometownOther>.+)/(?P<HighSchoolOther>.+)$')
        between['Hometown'] = between['Hometown'].combine_first(between['HometownOther'])
        between['HighSchool'] = between['HighSchool'].combine_first(between['HighSchoolOther'])
        data[['Hometown', 'High School']] = between[['Hometown', 'HighSchool']]
    elif 'Hometown / HS / Previous' in data.columns:
        data[['Hometown', 'High School', 'Previous']] = data['Hometown / HS / Previous'].str.split('/', expand = True)
    elif 'Hometown / High School / Previous' in data.columns:
        data[['Hometown', 'High School', 'Previous']] = data['Hometown / High School / Previous'].str.split('/', expand = True)    
    elif 'Hometown / Last School' in data.columns:
        data[['Hometown', 'High School']] = data['Hometown / Last School'].str.split('/', expand = True)
    elif 'Previous Team' in data.columns:
        data = data.rename(columns = {'Previous Team': 'High School'})
    elif 'Hometown / Last Team' in data.columns:
        data[['Hometown', "Other"]] = data['Hometown / Last Team'].str.split('/', n = 1, expand = True)
        if 'High School' not in data.columns:
            data['High School'] = data['Other']
    elif 'Ski Program' in data.columns:
        data['High School'] = data['Ski Program']
    elif 'Hometown/Previous Club' in data.columns:
        data[['Hometown', 'High School']] = data['Hometown/Previous Club'].str.split('/', expand = True)
    elif 'Hometown / Last Club' in data.columns:
        data[['Hometown', 'High School']] = data['Hometown / Last Club'].str.split('/', n = 1, expand = True)
    elif 'Hometown / Previous Team' in data.columns:
        data[['Hometown', 'High School']] = data['Hometown / Previous Team'].str.split('/', expand = True)
    elif 'Hometown / High School/ Previous School' in data.columns:
        data[['Hometown', 'High School']] = data['Hometown / High School/ Previous School'].str.split('/',n=1, expand = True)
    return data[['Name', 'Hometown', 'High School']]

In [40]:
# all_sports = []
# sports_1_year = []
page = BeautifulSoup(requests.get("https://bceagles.com/sports/baseball/schedule").content, 'html.parser')
all_sports = [a.get("href").split('/')[2] for a in page.find_all('a') if a.get('href') and 'roster' in a.get('href') and 'sports' in a.get('href') and 'https' not in a.get('href')]

In [90]:
def all_years(sport):
    base_link = f"https://bceagles.com/sports/{sport}/roster/?view=2"
    soup = BeautifulSoup(requests.get(f"https://bceagles.com/sports/{sport}/roster/").content, 'html.parser')
    links_to_visit = [a['value'] for a in soup.find(attrs = {'id':'ddl_past_rosters'}).find_all('option')]
    df = pd.DataFrame()
    for link in links_to_visit:
        year = link.split('/')[-1]
        if "201" in year or "202" in year:
            # print(year.split('-')[0])
            try:
                yearly = pd.DataFrame(get_roster(f"https://bceagles.com{link}"))
                yearly['Year'] = year.split('-')[0]
                df = pd.concat([df, yearly])
            except Exception as error:
                print(f"https://bceagles.com{link}", error)
    df['sport'] = sport
    return df


In [125]:
all_players = pd.DataFrame()
# Update Sports for school
for sport in all_sports:
    print(sport)
    all_players = pd.concat([all_players, all_years(sport)])

baseball
mens-basketball
womens-basketball
mens-cross-country
womens-cross-country
fencing
field-hockey
football
mens-golf
womens-golf
mens-ice-hockey
womens-ice-hockey
womens-lacrosse
womens-rowing
sailing
skiing
mens-soccer
womens-soccer
https://bceagles.com/sports/womens-soccer/roster/2024 list index out of range
https://bceagles.com/sports/womens-soccer/roster/2024 list index out of range
softball
swimming-and-diving
mens-tennis
womens-tennis
mens-track-and-field
https://bceagles.com/sports/mens-track-and-field/roster/2023-24 list index out of range
https://bceagles.com/sports/mens-track-and-field/roster/2023-24 list index out of range
womens-track-and-field
womens-volleyball
spirit-groups
https://bceagles.com/sports/spirit-groups/roster/2015-16 list index out of range
baseball


In [135]:
dupes_gone = all_players.drop_duplicates(subset = ['Name', 'Hometown', 'High School'])

In [136]:
dupes_gone.to_csv(f"All_{school}.csv")